In [8]:
import optuna
import mlflow
import mlflow.keras
from sklearn.metrics import accuracy_score

In [9]:
from data_handler import DataHandler

data_handler = DataHandler()
data_handler.load_data()
X_train, y_train, X_test, y_test = data_handler.preprocess_data()

Imbalance Rate Before Resampling: 19.418314572558565
Imbalance Rate After Resampling: 1.0074022632932969


In [15]:
from models.definition.cnn_lstm_model import CNNLSTMModel
from models.definition.lstm_model import LSTMModel
from models.definition.dense_model import DenseModel
from models.definition.gru_model import GRUModel
from models.definition.cnn_model import CNNModel
from models.definition.mlp_model import MLPModel

# input_shape = (X_train.shape[1], 1)  # or whatever the correct input shape is

# models = {
#     "CNN-LSTM": CNNLSTMModel(input_shape),
#     "LSTM": LSTMModel(input_shape),
#     "Dense": DenseModel(input_shape[0]),  # Assuming Dense model takes a different shape
#     "GRU": GRUModel(input_shape),
#     "CNN": CNNModel(input_shape),
#     "MLP": MLPModel(input_shape[0])  # Assuming MLP model takes a different shape
# }

In [17]:
from models.definition.cnn_lstm_model import CNNLSTMModel

def objective(trial):
    
    # conv1d_filters = trial.suggest_categorical('conv1d_filters', [32, 64, 128]),
    # conv1d_kernel_size = trial.suggest_categorical('conv1d_kernel_size', [3, 5, 7]),
    # lstm_units = trial.suggest_categorical('lstm_units', [64, 128, 256]),
    # activation = trial.suggest_categorical('activation', ['relu', 'tanh']),
    # dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    model = CNNLSTMModel(input_shape=X_train.shape[1:])

    # Train the model
    history = model.train_and_evaluate(
        X_train.reshape(X_train.shape[0],X_train.shape[1], -1), 
        y_train, 
        X_test.reshape(X_test.shape[0], X_test.shape[1], -1), 
        y_test, 
        'CNN-LSTM'
    )

    # Evaluate the model
    # For example, using accuracy or loss
    evaluation_metric = history.history['val_accuracy'][-1]

    # Logging the trial details to MLflow
    mlflow.log_params(trial.params)
    mlflow.log_metric('val_accuracy', evaluation_metric)

    return evaluation_metric

# Setup the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print study best trial
print('Best trial:', study.best_trial.params)

[I 2024-01-20 23:11:43,115] A new study created in memory with name: no-name-430021d0-bffe-4740-b66c-aa01a26a62dc
[W 2024-01-20 23:11:43,116] Trial 0 failed with parameters: {} because of the following error: TypeError("build_model() missing 1 required positional argument: 'input_shape'").
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\sieci_neuronowe\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_14928\2554761513.py", line 11, in objective
    model = CNNLSTMModel(input_shape=X_train.shape[1:])
  File "c:\Users\user\Desktop\sieci neuronowe\models\definition\cnn_lstm_model.py", line 8, in __init__
    #     self.input_shape = input_shape
TypeError: build_model() missing 1 required positional argument: 'input_shape'
[W 2024-01-20 23:11:43,120] Trial 0 failed with value None.


TypeError: build_model() missing 1 required positional argument: 'input_shape'

In [40]:
def objective(trial):
    # Choose the model and hyperparameters
    model_name = trial.suggest_categorical('model', ['CNN-LSTM', 'LSTM', 'Dense', 'GRU', 'CNN', 'MLP'])
    # model_class = models[model_name]
    # print('model_class', model_class)
    hyperparams = {}

    # print('model_name:', model_name,'\nmodel_class:', model_class )

    if model_name == "CNN-LSTM":
        hyperparams = {
            'conv1d_filters': trial.suggest_categorical('conv1d_filters', [32, 64, 128]),
            'conv1d_kernel_size': trial.suggest_categorical('conv1d_kernel_size', [3, 5, 7]),
            'lstm_units': trial.suggest_categorical('lstm_units', [64, 128, 256]),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }

    elif model_name == "CNN":
        hyperparams = {
            'conv1d_filters': trial.suggest_categorical('conv1d_filters', [32, 64, 128, 256]),
            'conv1d_kernel_size': trial.suggest_categorical('conv1d_kernel_size', [3, 5, 7]),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }

    elif model_name == "Dense":
        hyperparams = {
            'units': trial.suggest_categorical('units', [32, 64, 128]),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }

    elif model_name == "GRU":
        hyperparams = {
            'gru_units': trial.suggest_categorical('gru_units', [32, 64, 128, 256]),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }

    elif model_name == "LSTM":
        hyperparams = {
            'lstm_units': trial.suggest_categorical('lstm_units', [32, 64, 128, 256]),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }

    elif model_name == "MLP":
        hyperparams = {
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid']),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        }
        # hyperparams = {
        #     'dense_units': trial.suggest_categorical('dense_units', [64, 128, 256, 512]),
        #     'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid']),
        #     'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5)
        # }

    # if model_name == "Dense" or model_name == "MLP":
    #     model = model_class((X_train.shape[1],), **hyperparams)

    # # elif model_name == "GRU" or model_name == "CNN-LSTM":
    # #     model = model_class((X_train.shape[1], 1), **hyperparams)

    # else:
    #     model = model_class((X_train.shape[1], 1), **hyperparams)
    # model = model_class(**hyperparams)
        
    # if model_name in ["CNN-LSTM", "LSTM", "GRU", "CNN"]:
    #     model = model_class((X_train.shape[1], 1), **hyperparams)
    # elif model_name in ["Dense", "MLP"]:
    #     model = model_class((X_train.shape[1],), **hyperparams)
        
        # Get the model instance from the dictionary
    model_instance = models[model_name]

    # Update the model with hyperparameters
    model_instance.update_hyperparams(**hyperparams)   
    # Train the model
    model_instance.fit(X_train, y_train)
    # Train the model
    # model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model_instance.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Log with MLflow
    with mlflow.start_run():
        mlflow.log_params(hyperparams)
        mlflow.log_metric('accuracy', accuracy)

    return -accuracy  # Optuna tries to minimize the objective

# Set up MLflow
mlflow.set_tracking_uri("mlruns")
mlflow.set_experiment("DL_Models_Optimization_Experiment")

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Adjust the number of trials as needed

# Print the results
print("Best trial:")
trial = study.best_trial
print(f"  Value: {-trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-01-20 18:33:48,302] A new study created in memory with name: no-name-89dc8def-d71f-4f78-b3cc-e28604a716bd
[W 2024-01-20 18:33:48,305] Trial 0 failed with parameters: {'model': 'MLP', 'activation': 'relu', 'dropout_rate': 0.38052996447294973} because of the following error: AttributeError("'MLPModel' object has no attribute 'update_hyperparams'").
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\sieci_neuronowe\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_16940\2642076140.py", line 78, in objective
    model_instance.update_hyperparams(**hyperparams)
AttributeError: 'MLPModel' object has no attribute 'update_hyperparams'
[W 2024-01-20 18:33:48,306] Trial 0 failed with value None.


AttributeError: 'MLPModel' object has no attribute 'update_hyperparams'

In [6]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\user\Desktop\sieci neuronowe
